# [**Transformateur de colonne avec des sources de données hétérogènes**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_1_compose/plot_column_transformer.ipynb)<br/>([*Column Transformer with Heterogeneous Data Sources*](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer.html))

Les jeux de données peuvent souvent contenir des composants qui nécessitent des pipelines d'extraction et de traitement de caractéristiques différentes. Ce scénario peut se produire lorsque:
1. votre jeu de données est constitué de types de données hétérogènes (par exemple, des images matricielles et des légendes de texte),
2. votre jeu de données est stocké dans un [**`pandas.DataFrame`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) et différentes colonnes nécessitent des pipelines de traitement différents.

Cet exemple montre comment utiliser [**`ColumnTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) sur un jeu de données contenant différents types de caractéristiques. Le choix des caractéristiques n'est pas particulièrement utile, mais sert à illustrer la technique.

In [1]:
# Author: Matt Terry <matt.terry@gmail.com>
#
# License: BSD 3 clause

import numpy as np

from sklearn.preprocessing import FunctionTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC

# Jeu de données 20 newsgroups

Nous utiliserons le [**jeu de données 20 newsgroups** (7.2.2)](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset), qui comprend des publications de newsgroups sur 20 sujets. Ce jeu de données est divisé en sous-ensembles d'entraînement et de test en fonction de messages publiés avant et après une date spécifique. Nous n'utiliserons que des messages de 2 catégories pour accélérer le temps d'exécution.

In [2]:
categories = ["sci.med", "sci.space"]
X_train, y_train = fetch_20newsgroups(
    random_state=1,
    subset="train",
    categories=categories,
    remove=("footers", "quotes"),
    return_X_y=True,
)
X_test, y_test = fetch_20newsgroups(
    random_state=1,
    subset="test",
    categories=categories,
    remove=("footers", "quotes"),
    return_X_y=True,
)

Chaque caractéristique comprend des métadonnées sur cette publication, telles que le sujet et le corps du message d'actualités.

In [3]:
print(X_train[0])

From: mccall@mksol.dseg.ti.com (fred j mccall 575-3539)
Subject: Re: Metric vs English
Article-I.D.: mksol.1993Apr6.131900.8407
Organization: Texas Instruments Inc
Lines: 31




American, perhaps, but nothing military about it.  I learned (mostly)
slugs when we talked English units in high school physics and while
the teacher was an ex-Navy fighter jock the book certainly wasn't
produced by the military.

[Poundals were just too flinking small and made the math come out
funny; sort of the same reason proponents of SI give for using that.] 

-- 
"Insisting on perfect safety is for people who don't have the balls to live
 in the real world."   -- Mary Shafer, NASA Ames Dryden


# Création de transformateurs

Tout d'abord, nous voudrions un transformateur qui extrait le sujet et le corps de chaque publication. Comme il s'agit d'une transformation sans état (ne nécessite pas d'informations d'état provenant de données d'entraînement), nous pouvons définir une fonction qui effectue la transformation de données, puis utiliser [**`FunctionTransformer`**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) pour créer un transformateur scikit-learn.

In [4]:
def subject_body_extractor(posts):
    # construct object dtype array with two columns
    # first column = 'subject' and second column = 'body'
    features = np.empty(shape=(len(posts), 2), dtype=object)
    for i, text in enumerate(posts):
        # temporary variable `_` stores '\n\n'
        headers, _, body = text.partition("\n\n")
        # store body text in second column
        features[i, 1] = body

        prefix = "Subject:"
        sub = ""
        # save text after 'Subject:' in first column
        for line in headers.split("\n"):
            if line.startswith(prefix):
                sub = line[len(prefix) :]
                break
        features[i, 0] = sub

    return features


subject_body_transformer = FunctionTransformer(subject_body_extractor)

Nous allons également créer un transformateur qui extrait la longueur du texte et le nombre de phrases.

In [5]:
def text_stats(posts):
    return [{"length": len(text), "num_sentences": text.count(".")} for text in posts]


text_stats_transformer = FunctionTransformer(text_stats)

# Pipeline de classification

Le pipeline ci-dessous extrait le sujet et le corps de chaque publication en utilisant `SubjectBodyExtractor`, produisant un tableau `(n_samples, 2)`. Ce tableau est ensuite utilisé pour calculer des caractéristiques de sac de mots standard pour le sujet et le corps, ainsi que la longueur du texte et le nombre de phrases sur le corps, en utilisant `ColumnTransformer`. Nous les combinons, avec des poids, puis formons un classificateur sur l'ensemble combiné de caractéristiques.

In [6]:
pipeline = Pipeline(
    [
        # Extract subject & body
        ("subjectbody", subject_body_transformer),
        # Use ColumnTransformer to combine the subject and body features
        (
            "union",
            ColumnTransformer(
                [
                    # bag-of-words for subject (col 0)
                    ("subject", TfidfVectorizer(min_df=50), 0),
                    # bag-of-words with decomposition for body (col 1)
                    (
                        "body_bow",
                        Pipeline(
                            [
                                ("tfidf", TfidfVectorizer()),
                                ("best", TruncatedSVD(n_components=50)),
                            ]
                        ),
                        1,
                    ),
                    # Pipeline for pulling text stats from post's body
                    (
                        "body_stats",
                        Pipeline(
                            [
                                (
                                    "stats",
                                    text_stats_transformer,
                                ),  # returns a list of dicts
                                (
                                    "vect",
                                    DictVectorizer(),
                                ),  # list of dicts -> feature matrix
                            ]
                        ),
                        1,
                    ),
                ],
                # weight above ColumnTransformer features
                transformer_weights={
                    "subject": 0.8,
                    "body_bow": 0.5,
                    "body_stats": 1.0,
                },
            ),
        ),
        # Use a SVC classifier on the combined features
        ("svc", LinearSVC(dual=False)),
    ],
    verbose=True,
)

Enfin, nous ajustons notre pipeline sur les données d'entraînement et l'utilisons pour prédire les sujets de `X_test`. Les métriques de performance de notre pipeline sont alors imprimées.

In [7]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("Classification report:\n\n{}".format(classification_report(y_test, y_pred)))

[Pipeline] ....... (step 1 of 3) Processing subjectbody, total=   0.0s
[Pipeline] ............. (step 2 of 3) Processing union, total=   1.2s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.0s
Classification report:

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       396
           1       0.87      0.83      0.85       394

    accuracy                           0.85       790
   macro avg       0.85      0.85      0.85       790
weighted avg       0.85      0.85      0.85       790

